#Lab 4
Cameron Burdsall

In [1]:
# Imports
import numpy as np
import pandas as pd
import math, random
from sklearn.utils import shuffle

In [2]:
# training set
train_df = pd.read_csv(
    filepath_or_buffer='http://www.cse.scu.edu/~yfang/coen140/crime-train.txt', sep='\t') #tab separated
# test set
test_df = pd.read_csv(
    filepath_or_buffer='http://www.cse.scu.edu/~yfang/coen140/crime-test.txt', sep='\t')

In [3]:
# Preprocess data by arranging columns

#declare matrix X and insert Column of 1's into the data frame
df = train_df
train_df.T
#take target column
y_column = 'ViolentCrimesPerPop'
y = df[y_column]

#create a copy of the df, drop the y col, and add a col full of 1's
X = df.copy()
X = X.drop(y_column, axis = 1)
X.insert(0, 'ones', 1)

#create testing datasets
df = test_df
y_test = df[y_column]
X_test = df.copy()
X_test = X_test.drop(y_column, axis = 1)
X_test.insert(0, 'ones', 1)

In [4]:
def RMSE (prediction, result):
  # input are two vectors, one holding the generated prediction, one the actual result
  # caluclate difference between vectors
  dif = prediction - result
  tot = 0
  # go through and sum the squares of the differences
  for num in dif:
    tot += num ** 2
  return math.sqrt(tot / len(dif))

#Linear Regression Closed Form
Using the closed form property that **b = (X\`X)^-1(X\`Y)** where b is the coefficient vector,
X is the independent variables matrix, X` is the transpose of X, and Y is the solution vector

In [5]:
def problem1 (train, y_train, predict):
  # train variables used to construct model, predict used to set what model is then predicting
  # predict theta parameters and return the result of the dot product of the dataset and the betas

  # dealing with the left parenthesis in the coefficient equation above
  lefthand = np.dot(train.T, train)
  lefthand = np.linalg.inv(lefthand)

  # dealing with the right parenthesis
  righthand = np.dot(train.T, y_train)

  #calculate coefficient predictions
  betas = np.dot(lefthand, righthand)

  # apply betas to all entries in the set which we want to predict
  return np.dot(predict, betas)

In [6]:
# find RMSE on original set
train_prediction = RMSE(problem1(X, y, X), y)
print(train_prediction)

# find RMSE on testing set
test_prediction = RMSE(problem1(X, y, X_test), y_test)
print(test_prediction)

0.12768967421762206
0.14583464490948847


In [7]:
def generate_permutation (n):
  lst=[]
  for i in range(n):
    lst.append(i)
  random.shuffle(lst)
  return lst

#Stochastic Gradient Decent



In [11]:
def generate_permutation(n):
  a = []
  for i in range (n):
    a.append(i)
  a = shuffle(a)
  return a
def SGD(X, y, nullbias = False, maxit=1e+6, alpha=5e-4, threshold=10e-10):
  #default alpha and threshold values are the ones that were found
  #to achieve the lowest RMSE scores

  #random initial weights
  t = np.random.randn(len(X.columns), 1)

  #null bias
  if nullbias:
    t[0] = 0 
  new_t = None

  #generate initial permutation
  #perm = generate_permutation(len(X))
  it = 0
  sdif = 1

  while (abs(sdif) > threshold) and (it < maxit):
    # take a step for each sample in a random permutation of the dataset
    # reshuffle the permutation when we have gone thru the entire set
    # evaluate for theta for each entry in the permutation to find a model using Stochatic Gradient Decent
    #for ind in perm:
    ind = random.randint(0, len(X) - 1)
      #grab samples for this iteration
    X_entry = X.iloc[[ind]]
    y_entry = y.iloc[[ind]].item()

      #formatting due to errors
    y_entry = np.asarray(y_entry)
    y_entry.reshape(1, 1) 

      #apply thetas to the entry to yield the estimated value
    y_hat = np.dot(X_entry, t)

      #calculate the cost
    cost = y_entry - y_hat
      # (y_hat - y)xi
    cost_entry = np.dot(X_entry.T, cost)
      #new_theta = theta + (alpha(y_hat - y)xi)
    new_t = t + (alpha * cost_entry)
      #calculate theta difference to check for convergence
    dif = new_t - t
    sdif = np.sum(dif)

      #delete the comment below to see how the model converges, but this will make the program take MUCH more time to execute due to print overhead
      #print(sdif)

      #assign new thetas
    t = new_t
    it += 1
      # check for convergence
    if (abs(sdif) < threshold):
      print('Convergence Reached')
      print('Iterations: ' + str(it))
      break
      # if we reached max iterations
    if (maxit <= it):
      print('No convergence')
      print('Max Iterations Reached: ' + str(int(maxit)))
      #return last model if max iterations reached
      return t
    #perm = shuffle(perm)
  #return model generated by SGD
  return t

model = SGD(X, y)

IndexError: positional indexers are out-of-bounds

In [10]:
#having alpha be greater than 5e-3 causes the loss score to somehow INCREASE

#apply model to the dataset
y_pred = np.dot(X, model)
y_tes = np.dot(X_test, model)

#formatting nonsense
t_y = np.asarray(y).reshape(1595, 1)
tt_y = np.asarray(y_test).reshape(399, 1)

#RMSE scores vary wildly, model generation is not consistent
train_RMSESGD = RMSE(y_pred, t_y)
print(train_RMSESGD)
test_RMSESGD = RMSE(y_tes, tt_y)
print(test_RMSESGD)

NameError: name 'model' is not defined

#Best Stochatic Gradient Model Decent Parameters & Results
1.  alpha = 5e-4 <br>
    threshold = 10e-10 <br>
    Train RMSE: 0.14196462300010157 <br>
    Test RMSE: 0.162137795588426 <br>
    Iterations: 707865 <br>
2.  alpha = 5e-4 <br>
    threshold = 10e-10 <br>
    Train RMSE: 0.14279185205207573 <br>
    Test RMSE: 0.15955539536691501 <br>
    Iterations: 834326 <br>
3.  alpha = 5e-3 <br>
    threshold = 10e-9 <br>
    Train RMSE: 0.14445652339078446 <br>
    Test RMSE: 0.16299297230777626 <br>
    Iterations: 301456 <br>

 

I noticed that typically, more iterations led to a more accurate model, except for the cases where the model gets stuck in local minima. This would occur if the alpha and threshold were set sufficeintly low. The best step values were in the range of 5e-3 to 5e-4. Anything above 5e-3 would overshoot and cause the model generation to get stuck in an infinite loop with the model values jumping around wildly, and anything smaller than 5e-4 would cause the thetas to get stuck in a local minima and not find the global. Generally, the lower the threshold, the better the results, except when the theshold is set too so such that the GD never converges, and oscillates around the minimum infinitely.<br><br>

The Closed form solution was able to genearate a more accurate as well as consistent model much faster than using SGD<br>
